In [1]:
import os
os.environ["OPENAI_API_KEY"] = input("Enter OPENAI_API_KEY:")

In [2]:
import weave
import json
dataset="set1_test"
weave.init(f"highly-biased/test_final_v2")
dataset_ref  = weave.ref("set1_test").get()


weave version 0.51.28 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: xp20-hb.
View Weave data at https://wandb.ai/highly-biased/test_final_v2/weave


In [18]:
model = "gpt-4o-mini-2024-07-18"
model = "gpt-4o-2024-11-20"
# model = "gpt-3.5-turbo-0125"

In [8]:
jsonl = []

for row in dataset_ref.rows:
    jsonl.append(
        {"custom_id": model + "-" + dataset + "-" + row["id"],
         "method": "POST", 
         "url": "/v1/chat/completions",
         "body":{
            "model":model,
            "messages":[
                {
                "role": "developer",
                "content": row["system_prompt"]
                },
                {
                "role": "user",
                "content": row["user_prompt"]
                }            
            ],
            "max_tokens": 2000
         }}
    )
with open(f"openai-batch-files/{dataset_ref.name}-{model}.jsonl", 'w') as f:
    for item in jsonl:
        f.write(json.dumps(item) + "\n")

**Upload the generated file to OpenAI and start a batch from the web interface**  
(This is not done here, to prevent accidental execution)

In [4]:
from openai import OpenAI
client = OpenAI()

Check status of Batch request:

In [22]:
batch_name = "batch_6787c7331d8c8190afa976e613f1ffa5"

In [23]:
batch_run = client.batches.retrieve(batch_name)
batch_run

Batch(id='batch_6787c7331d8c8190afa976e613f1ffa5', completion_window='24h', created_at=1736951603, endpoint='/v1/chat/completions', input_file_id='file-6Kf2UDRpXi2KHukEifU3GA', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1736952074, error_file_id=None, errors=None, expired_at=None, expires_at=1737038003, failed_at=None, finalizing_at=1736951980, in_progress_at=1736951604, metadata={}, output_file_id='file-5bSL8ps9qGQWoVPCG9YHBv', request_counts=BatchRequestCounts(completed=720, failed=0, total=720))

In [24]:
file_response = client.files.content(batch_run.output_file_id)
with open(f"openai-batch-files/output/{dataset_ref.name}-{model}-{batch_name}.jsonl", 'w') as f:
    f.write(file_response.text)

## jsonl to pandas

In [47]:
import pandas as pd

In [79]:
gpt_out_df = pd.read_json(f"openai-batch-files/output/{dataset_ref.name}-{model}-{batch_name}.jsonl", lines=True)
gpt_out_df = pd.concat((gpt_out_df[["id","custom_id"]],
                        pd.json_normalize(gpt_out_df["response"]).add_prefix("response.")[["response.body.choices", 
                                                                                           "response.body.usage.prompt_tokens", 
                                                                                           "response.body.usage.completion_tokens",
                                                                                           "response.body.usage.total_tokens",
                                                                                           "response.body.model"
                                                                                           ]]), 
                        axis="columns")
gpt_out_df["content"] = gpt_out_df["response.body.choices"].apply(lambda response: response[0]["message"]["content"])
gpt_out_df

,id,custom_id,response.body.choices,response.body.usage.prompt_tokens,response.body.usage.completion_tokens,response.body.usage.total_tokens,response.body.model,content
0,batch_req_6786bd099e1c8190b9432a2bc01c3eae,gpt-4o-mini-2024-07-18-set1_test-0,"[{'index': 0, 'message': {'role': 'assistant',...",57,1042,1099,gpt-4o-mini-2024-07-18,1. **Albert Einstein**:\n Albert Einstein (1...
1,batch_req_6786bd09b2c48190a8d9453b65dc1f05,gpt-4o-mini-2024-07-18-set1_test-1,"[{'index': 0, 'message': {'role': 'assistant',...",57,864,921,gpt-4o-mini-2024-07-18,### Albert Einstein\nAlbert Einstein was a the...
2,batch_req_6786bd09c7a88190beb002126d09d14b,gpt-4o-mini-2024-07-18-set1_test-2,"[{'index': 0, 'message': {'role': 'assistant',...",57,1094,1151,gpt-4o-mini-2024-07-18,"### 1. Albert Einstein\nAlbert Einstein, born ..."
3,batch_req_6786bd09e108819091a3355ae291a8a5,gpt-4o-mini-2024-07-18-set1_test-3,"[{'index': 0, 'message': {'role': 'assistant',...",57,1103,1160,gpt-4o-mini-2024-07-18,Sure! Here’s a detailed description of each it...
